In [187]:
## Set up Conda environment and attach copilot-playground kernel to this notebook

# pip install --user ipykernel
# conda create -n copilot-playground
# conda activate copilot-playground

# conda install ipykernel
# conda install postgresql
# conda install psycopg2
# conda install matplotlib
# conda install plotly
# conda install scipy
# conda install sklearn
# conda install langchain
# conda install openai
# conda install python-dotenv

# conda install postgresql
# conda install psycopg2

In [188]:
## Set up .env file in the same directory as this notebook, with the following 3 variables

import os
from dotenv import load_dotenv

load_dotenv(".env")

open_ai_api_key = os.getenv("OPEN_AI_API_KEY")
normandy_db_username = os.getenv("DB_USER_NAME")
normandy_db_local_port = os.getenv("DB_PORT")
normandy_api_username = os.getenv("NORMANDY_API_USERNAME")
normandy_api_password = os.getenv("NORMANDY_API_PW")

print(normandy_db_username)
print(normandy_db_local_port)

aditya.gunda@getathelas.com
54329


In [189]:
# Authenticate with Normandy API - https://rcm-api.athelas.com/api/docs#/authentication/login_for_access_token_v1_auth_token_post

BASE_URL = "https://rcm-api.athelas.com/v1/"

import requests

def get_access_token():
  # The URL you want to make a request to
  url = BASE_URL + "auth/token"

  # The form data you want to send
  data = {
      "username": normandy_api_username,
      "password": normandy_api_password
  }

  response = requests.post(url, data=data)
  json = response.json()
  access_token = json["access_token"]

  return access_token

ACCESS_TOKEN = get_access_token()

In [190]:
import psycopg2

# NOTE: Make sure that you have the Cloud SQL Auth Proxy running locally for Normandy DB at the provided port.

def connect_to_db():
    try:
        connection = psycopg2.connect(
            user=normandy_db_username,
            password="",
            host="localhost",
            port=normandy_db_local_port,
            database="normandy"
        )
        return connection
    except Exception as error:
        print(f"Error while connecting to PostgreSQL: {error}")

def fetch_data(query):
    connection = connect_to_db()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        records = cursor.fetchall()
        return records
    except Exception as error:
        print(f"Error fetching data: {error}")
    finally:
        cursor.close()
        connection.close()


In [191]:
import json 
def lookup_claim_submission_payload(claim_id: int):
  claim_payload = fetch_data(f"SELECT submission_request FROM claim_submissions WHERE id = {claim_id};")[0]
  claim_payload = claim_payload[0]
  return json.dumps(claim_payload, indent=4)

print(lookup_claim_submission_payload(claim_id=617417))

{
    "billing": {
        "npi": "1699448316",
        "address": {
            "city": "MESA",
            "state": "AZ",
            "address1": "1 N MACDONALD STE 204",
            "postalCode": "852017343"
        },
        "employerId": "853606335",
        "providerType": "BillingProvider",
        "taxonomyCode": "363LP0808X",
        "organizationName": "Justin McKinley, PMHNP DBA Blu"
    },
    "receiver": {
        "organizationName": "AETNA"
    },
    "rendering": {
        "npi": "1053907840",
        "address": {
            "city": "MESA",
            "state": "AZ",
            "address1": "1 N MACDONALD STE 204",
            "postalCode": "852017343"
        },
        "lastName": "McKinley",
        "firstName": "Justin",
        "employerId": "853606335",
        "providerType": "RenderingProvider",
        "taxonomyCode": "363LP0808X"
    },
    "submitter": {
        "organizationName": "Justin McKinley, PMHNP DBA Blue Water Psychiatry",
        "contactInformati

In [192]:
# TODO: add npi registry tool

print(requests.get("https://npiregistry.cms.hhs.gov/api/?version=2.1", params={"number":1952413973 }).text)

{"result_count":1,"results":[{"created_epoch":"1157040685000","enumeration_type":"NPI-1","last_updated_epoch":"1183964604000","number":"1952413973","addresses":[{"country_code":"US","country_name":"United States","address_purpose":"MAILING","address_type":"DOM","address_1":"1380 LUSITANA ST","address_2":"SUITE 608","city":"HONOLULU","state":"HI","postal_code":"968132421","telephone_number":"808-536-2261","fax_number":"808-538-3957"},{"country_code":"US","country_name":"United States","address_purpose":"LOCATION","address_type":"DOM","address_1":"1380 LUSITANA ST","address_2":"SUITE 608","city":"HONOLULU","state":"HI","postal_code":"968132421","telephone_number":"808-536-2261","fax_number":"808-538-3957"}],"practiceLocations":[],"basic":{"first_name":"JEFFERY","last_name":"HARPSTRITE","middle_name":"KIMO","credential":"MD","sole_proprietor":"NO","gender":"M","enumeration_date":"2006-08-31","last_updated":"2007-07-09","status":"A","name_prefix":"Dr.","name_suffix":"--"},"taxonomies":[{"c

In [217]:
import json

def lookup_claim_denial_info(claim_id: int):
  def map_data_to_dicts(data):
    keys = [
        "claim_submission_id", "cpt_code", "modifiers", "place_of_service_code",
        "revenue_code", "insurance_payout", "smartpay_pr", 
        "claim_adjustment_reason_codes", "remittance_advice_remark_codes"
    ]
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
          WITH payout AS (
            SELECT
                csp.claim_submission_id,
                csp.procedure_id,
                csp.modifiers,
                v.remit_insurance_priority::text,
                SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float / 100 as insurance_payout,
                SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float / 100 as smartpay_pr
            FROM
                claim_submissions_procedures csp
            JOIN
                all_procedure_details_view v ON (
                            v.claim_submission_id = csp.claim_submission_id
                        AND v.procedure_id = csp.procedure_id
                    )
            WHERE
                csp.claim_submission_id = '{claim_id}'
          GROUP BY 1,2,3,4
        )
        , carc_rarc AS (
          SELECT
              pr.procedure_id,
              ARRAY_AGG(CONCAT(carc.code,' - ', carc.description)) as claim_adjustment_reason_codes,
              ARRAY_AGG(CONCAT(rarc.code, ' - ', rarc.description)) as remittance_advice_remark_codes
          FROM
                  claim_submissions cs
          LEFT JOIN
                  claim_reconciliations crecon ON cs.id = crecon.claim_submission_id
          LEFT JOIN
                  claim_remittances rem ON rem.claim_reconciliation_id = crecon.id
          LEFT JOIN
                  procedure_remittances pr ON pr.claim_remittance_id = rem.id
          LEFT JOIN
                  remittance_advice_remark_codes_procedure_remittances rarcpr ON rarcpr.procedure_remittances_id = pr.id
          LEFT JOIN
                  remittance_advice_remark_codes rarc ON rarc.code = rarcpr.remittance_advice_remark_code
          LEFT JOIN
                  procedure_adjustments pa ON pa.procedure_remittance_id = pr.id
          LEFT JOIN
                  claim_adjustment_reason_codes carc ON carc.code = pa.claim_adjustment_reason_code
          WHERE
              cs.id = '{claim_id}'
              AND carc.code NOT IN ('45','1','2','3','253','100','97','18','118','23','B11')
          GROUP BY
              1
        )

        SELECT
            payout.claim_submission_id,
            cc.code as cpt_code,
            payout.modifiers,
            proc.place_of_service_code,
            rc.code as revenue_code,
            payout.insurance_payout,
            payout.smartpay_pr,
            cr.claim_adjustment_reason_codes,
            cr.remittance_advice_remark_codes,
            e.from_date_range as start_service_date_range,
            e.to_date_range as end_service_date_range,
            e.id as encounter_id,
            p.id as rendering_provider_id,
            p.first_name as rendering_provider_first_name,
            p.last_name as rendering_provider_last_name,
            p.individual_npi as rendering_provider_npi
        FROM
            payout
        JOIN
            procedures proc ON proc.id = payout.procedure_id
        JOIN
            cpt_codes cc ON cc.id = proc.cpt_code_id
        LEFT JOIN
            revenue_codes rc ON proc.revenue_code_id = rc.id
        LEFT JOIN
            carc_rarc cr ON cr.procedure_id = proc.id
        LEFT JOIN
            encounters e ON e.id = proc.encounter_id
        LEFT JOIN
            providers p ON p.id = e.rendering_provider_id
    """.format(claim_id=claim_id)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)


In [215]:
lookup_claim_denial_info(claim_id=617417)

'[\n    {\n        "claim_submission_id": 617417,\n        "cpt_code": "99214",\n        "modifiers": [\n            "95",\n            "25"\n        ],\n        "place_of_service_code": "11",\n        "revenue_code": null,\n        "insurance_payout": 0.0,\n        "smartpay_pr": 130.0,\n        "claim_adjustment_reason_codes": [\n            "27 - Expenses incurred after coverage terminated."\n        ],\n        "remittance_advice_remark_codes": [\n            "N650 - This policy was not in effect for this date of loss. No coverage is available."\n        ]\n    },\n    {\n        "claim_submission_id": 617417,\n        "cpt_code": "90836",\n        "modifiers": [\n            "95"\n        ],\n        "place_of_service_code": "11",\n        "revenue_code": null,\n        "insurance_payout": 0.0,\n        "smartpay_pr": 130.0,\n        "claim_adjustment_reason_codes": [\n            "27 - Expenses incurred after coverage terminated."\n        ],\n        "remittance_advice_remark_co

In [195]:
import json

def lookup_provider_taxonomy_codes(provider_id: int | str):
  def map_data_to_dicts(data):
    keys = [
        "code", "description", "is_primary"
    ]
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
          SELECT code, description, is_primary from taxonomies where provider_id = {provider_id};
    """.format(provider_id=provider_id)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)


In [196]:
import json 

def lookup_approved_claims_with_cpt_code(cpt_code: str):
  def map_data_to_dicts(data):
    keys = [
        "claim_submission_id", "cpt_codes", "place_of_service_code",
        "revenue_code", "insurance_payout", "smartpay_pr"
    ]
    
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
    WITH relevant_claims AS (
      SELECT
          ARRAY_AGG(DISTINCT claim_submission_id) as claim_submission_ids
      FROM
          claim_submissions_procedures csp
      JOIN
        procedure_payout_view ppv ON ppv.procedure_id = csp.procedure_id
      JOIN
        cpt_codes cc ON csp.cpt_code_id = cc.id
      WHERE
          (cc.code = '{cpt_code}')
          AND COALESCE(ppv.provider_paid_amount_in_usd_cents, 0) > 0
    ),
    payout AS (
      SELECT
          csp.claim_submission_id,
          csp.procedure_id,
          csp.modifiers,
          v.remit_insurance_priority::text,
          SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float / 100 as insurance_payout,
          SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float / 100 as smartpay_pr
      FROM
          claim_submissions_procedures csp
      JOIN
          all_procedure_details_view v ON (
                  v.claim_submission_id = csp.claim_submission_id
                  AND v.procedure_id = csp.procedure_id
              )
      WHERE
        csp.claim_submission_id IN (SELECT unnest(claim_submission_ids) FROM relevant_claims)
      GROUP BY 1,2,3,4
      HAVING SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float > 0 OR SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float > 0
      ORDER BY 1 DESC
    ) 

    SELECT
        payout.claim_submission_id,
        ARRAY_AGG(CONCAT(cc.code, '-', payout.modifiers)) as cpt_codes,
        ARRAY_AGG(DISTINCT proc.place_of_service_code) as place_of_service_code,
        ARRAY_AGG(DISTINCT rc.code) as revenue_code,
        SUM(COALESCE(payout.insurance_payout, 0)) as insurance_payout,
        SUM(COALESCE(payout.smartpay_pr, 0)) as smartpay_pr
    FROM
        payout
    JOIN
        procedures proc ON proc.id = payout.procedure_id
    JOIN
        cpt_codes cc ON cc.id = proc.cpt_code_id
    LEFT JOIN
        revenue_codes rc ON proc.revenue_code_id = rc.id
    GROUP BY
      1
    LIMIT 50;
  """.format(cpt_code=cpt_code) 
    # modifiers_str=','.join(f"'{item}'" for item in modifiers)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)

In [197]:
# lookup_approved_claims_with_cpt_code(cpt_code="99213")

In [198]:
import json 

def lookup_approved_claims_with_modifier(modifier: str  | int):
  if (isinstance(modifier, str)):
    modifier = modifier.upper()

  def map_data_to_dicts(data):
    keys = [
        "claim_submission_id", "cpt_codes", "place_of_service_code",
        "revenue_code", "insurance_payout", "smartpay_pr"
    ]
    
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
    WITH relevant_claims AS (
      SELECT
          ARRAY_AGG(DISTINCT claim_submission_id) as claim_submission_ids
      FROM
          claim_submissions_procedures csp
      JOIN
        procedure_payout_view ppv ON ppv.procedure_id = csp.procedure_id
      WHERE
          ('{modifier}' = ANY(csp.modifiers))
          AND COALESCE(ppv.provider_paid_amount_in_usd_cents, 0) > 0
    ),
    payout AS (
      SELECT
          csp.claim_submission_id,
          csp.procedure_id,
          csp.modifiers,
          v.remit_insurance_priority::text,
          SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float / 100 as insurance_payout,
          SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float / 100 as smartpay_pr
      FROM
          claim_submissions_procedures csp
      JOIN
          all_procedure_details_view v ON (
                  v.claim_submission_id = csp.claim_submission_id
                  AND v.procedure_id = csp.procedure_id
              )
      WHERE
        csp.claim_submission_id IN (SELECT unnest(claim_submission_ids) FROM relevant_claims)
      GROUP BY 1,2,3,4
      HAVING SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float > 0 OR SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float > 0
      ORDER BY 1 DESC
    ) 

    SELECT
        payout.claim_submission_id,
        ARRAY_AGG(CONCAT(cc.code, '-', payout.modifiers)) as cpt_codes,
        ARRAY_AGG(DISTINCT proc.place_of_service_code) as place_of_service_code,
        ARRAY_AGG(DISTINCT rc.code) as revenue_code,
        SUM(COALESCE(payout.insurance_payout, 0)) as insurance_payout,
        SUM(COALESCE(payout.smartpay_pr, 0)) as smartpay_pr
    FROM
        payout
    JOIN
        procedures proc ON proc.id = payout.procedure_id
    JOIN
        cpt_codes cc ON cc.id = proc.cpt_code_id
    LEFT JOIN
        revenue_codes rc ON proc.revenue_code_id = rc.id
    GROUP BY
      1
    LIMIT 50;
  """.format(modifier=modifier) 
    # modifiers_str=','.join(f"'{item}'" for item in modifiers)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)

In [199]:
# lookup_approved_claims_with_modifier(modifier="KX")

In [200]:
import json 

def get_cpt_code_description(cpt_code: str):
  def map_data_to_dicts(data):
    keys = [
        "cpt_code", 
        "description"
    ]
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
    SELECT code as cpt_code, description FROM cpt_codes WHERE code = '{cpt_code}';
    """.format(cpt_code=cpt_code))

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)

In [201]:
import pandas as pd
import json 

def get_definition_for_modifier(modifier: str):
  if not isinstance(modifier, str):
    modifier = str(modifier)
  modifier = modifier.upper()

  df = pd.read_csv("./all_modifiers.csv")
  df = df[df["Modifier"] == modifier]
  return json.dumps(json.loads(df.to_json(orient="records")), indent=4)

In [202]:
import json
from langchain import OpenAI

def service_line_sort_key(service_line):
  if 'institutionalService' in service_line:
    return (
      service_line['institutionalService'].get('revenueCode', ''),
      service_line['institutionalService'].get('serviceDate', '')
    )
  else:
    return (
        service_line['professionalService']['procedureCode'],
        service_line['professionalService'].get('modifiers', ''),  # Assuming 'modifiers' might not always be present
        service_line.get('serviceDate', '')
    )

def sorted_service_lines(json_str):
    data = json.loads(json_str)
    if 'claimInformation' not in data or 'serviceLines' not in data['claimInformation']:
        return data
    data['claimInformation']['serviceLines'] = sorted(
        data['claimInformation']['serviceLines'],
        key=service_line_sort_key
    )
    return data

def dict_diff(d1, d2, parent_key=''):
    d1 = sorted_service_lines(json.dumps(d1))
    d2 = sorted_service_lines(json.dumps(d2))
    differences = {}
    for k, v in d1.items():
        if k not in d2:
            differences[k] = {'from': v, 'to': None}
        else:
            if isinstance(v, dict):
                inner_diff = dict_diff(v, d2[k], parent_key=f'{parent_key}.{k}' if parent_key else k)
                if inner_diff:
                    differences[k] = inner_diff
            elif v != d2[k]:
                differences[k] = {'from': v, 'to': d2[k]}
    
    for k, v in d2.items():
        if k not in d1:
            differences[k] = {'from': None, 'to': v}

    return differences

def get_claim_payload_diffs_for(list_of_claims: list[int] | str):
  if isinstance(list_of_claims, str):
    list_of_claims = json.loads(list_of_claims)
  
  payloads = fetch_data("""
      WITH selected_claims AS (
        SELECT 
          *
        FROM 
          claim_submissions cs
        WHERE 
          cs.id = ANY(ARRAY[{all_claims_string}])
      )
      
      SELECT DISTINCT ON (cs.encounter_id)
        cs.encounter_id as encounter_id,
        cs.id as prev_claim_id,
        cs.submission_request as prev_claim_payload,
        sc.id as resubmitted_claim_id,
        sc.submission_request as resubmitted_claim_payload
      FROM claim_submissions cs
      JOIN selected_claims sc ON sc.encounter_id = cs.encounter_id
      WHERE cs.id < sc.id and cs.status = 'SUBMITTED'
      ORDER BY cs.encounter_id, cs.id DESC;
  """.format(all_claims_string=', '.join(f'{i}' for i in list_of_claims))
  )

  def map_data_to_dicts(data):
    keys = [
        "encounter_id", "prev_claim_id", "prev_claim_payload",
        "resubmitted_claim_id", "resubmitted_claim_payload"
    ]
    
    return [dict(zip(keys, row)) for row in data]


  if payloads is None:
    return json.dumps([])

  payloads = map_data_to_dicts(payloads)

  condensed_payloads = []
  for entry in payloads:
      cp = {
        'diff': dict_diff(entry['prev_claim_payload'], entry['resubmitted_claim_payload']),
        'encounter_id': entry['encounter_id'],
        'resubmitted_claim_id': entry['resubmitted_claim_id']
      }
      condensed_payloads.append(cp)

  # Limit to 15 claims for now
  condensed_payloads = condensed_payloads[:15]

  str_payload = json.dumps(condensed_payloads, indent=4)

  diff_sub_prompt = """
    Resubmitted Claim ID: {resubmitted_claim_id}
    Encounter ID: {encounter_id}
    Diff: {diff}

  """
  diff_section = '\n'.join(diff_sub_prompt.format(**cp) for cp in condensed_payloads)

  prompt = """
    {diff_section}

    Instructions: You are a medical biller trying to understand the changes made between two claims. Summarize the diff made betwen the previous and resubmitted claim in 2-3 sentences maximum. Always include the resubmitted claim's ID in the explanation. Ignore changes to the patient control number, claim control number, and frequency code. \n"

  """.format(diff_section=diff_section)

  llm = OpenAI(temperature=0, model_name="gpt-4-32k")
  
  return llm(prompt)
  


In [203]:
# print(get_claim_payload_diffs_for([72849,161706,560167]))

In [204]:
import requests

def check_patient_eligibility(patient_id: int):
  response = requests.get(BASE_URL+"eligibility/check_patient_eligibility/{patient_id}".format(patient_id=patient_id), headers={"Authorization": f"Bearer {ACCESS_TOKEN}".format(ACCESS_TOKEN=ACCESS_TOKEN)})
  return json.dumps(response.json(), indent=4)
  
def get_all_patient_insurance_info(patient_id: int):
  response = requests.get(BASE_URL+"patient_insurance_info/by_patient_id", params={ 'patient_id': patient_id }, headers={"Authorization": f"Bearer {ACCESS_TOKEN}".format(ACCESS_TOKEN=ACCESS_TOKEN)})
  return json.dumps(response.json(), indent=4)

def summarize_patient_insurance_history_for_claim(claim_id: int | str):
  patient_id = fetch_data("""
    SELECT encounter.patient_id
    FROM claim_submissions cs
    JOIN encounters encounter ON encounter.id = cs.encounter_id
    WHERE cs.id = '{claim_id}';
  """.format(claim_id=claim_id)
  )[0][0]

  patient_insurance_info = get_all_patient_insurance_info(patient_id)
  patient_eligbility = check_patient_eligibility(patient_id)
  prompt = """
    All Patient Insurance Info:
    {patient_insurance_info}

    Latest Patient Eligibility:
    {patient_eligbility}

    Instructions: You are a medical biller trying to understand the history of a patient's policy/coverage. Summarize the patient's insurance history and include relevant dates and policy numbers when available. \n
  """.format(patient_insurance_info=patient_insurance_info, patient_eligbility=patient_eligbility)

  llm = OpenAI(temperature=0, model_name="gpt-4-32k")
  return llm(prompt)

In [205]:
from langchain.agents import Tool

shared_agent_tools = [
      Tool(
        name="Lookup Provider's Taxonomy Codes",
        func=lookup_provider_taxonomy_codes,
        description="""
          Useful for when you need to lookup a provider's taxonomy codes.
          Taxonomy codes are used to classify the type, classification, and/or specialization of health care providers.

          Args:
            provider_id (int): the id of the provider you want to lookup
          Returns:
            list of dicts (str): Each dict will have the following keys: "code", "description", "is_primary"
        """
      ),
      Tool(
        name="Lookup Claim Submission Payload",
        func=lookup_claim_submission_payload,
        description="""
          Useful for when you need to lookup the payload of a claim submission.
          
          Args:
            claim_id (int): the id of the claim you want to lookup
          Returns:  
            str: the claim submission payload
        """,
      ),
        Tool(
        name="Lookup Changes Made to Resubmitted Claim Payload",
        func=get_claim_payload_diffs_for,
        description="""
          Useful for when you need to find out what was changed in a resubmitted claim's payload.

          Args:
            list_of_claims (list[int]): a list of claim ids you want to search for (e.g. [72849,161706,560167])
          Returns:
            str: summary of the changes made to many of the provided resubmitted claims
        """,
    ),
    Tool(
        name="Lookup Claim Procedure Denial Info",
        func=lookup_claim_denial_info,
        description="""
          Useful for when you need get a claim's cpt codes and their denial reasons.
          Claims may have multiple procedures.
          The denial reasons are provided in the form of claim adjustment reason codes and remittance advice remark codes.

           Args: 
            claim_id (int): the id of the claim you want to lookup
          Returns:
            list[tuple]: A list containing tuples with procedure data. Each tuple  follows this format: (claim_submission_id, cpt_code, modifiers, place_of_service_code, revenue_code, insurance_payout, smartpay_pr, claim_adjustment_reason_codes, remittance_advice_remark_codes)
        """,
    ),
    Tool(
        name="Get CPT Code Description",
        func=get_cpt_code_description,
        description="""
          Useful to find the description of a cpt code.
          
          Args: 
           cpt_code (str): the cpt code you want to search for (e.g. 99214)
          Returns:
            list[tuple]: A list containing tuples with the cpt code and description. Each tuple follows this format: (cpt_code, description)
         """,
    ),
     Tool(
        name="Get Definition for Modifier",
        func=get_definition_for_modifier,
        description="""
          Useful for when you need to lookup the definition of a modifier.

          Args:
            modifier (str): the modifier you want to lookup (e.g. 90)
          Returns:
            list of dicts (str): Each dict will have the following keys: "Modifier", "Definition"
        """,
    ),
    Tool(
        name="Lookup Approved Claims where CPT Code was approved",
        func=lookup_approved_claims_with_cpt_code,
        description="""
          Useful to find claims where a certain cpt_code was approved. 
          You can use the claim_submission_id to determine which procedures that are in the same claim.
          
          Args: 
            cpt_code (str): the cpt code you want to search for (e.g. 99214)
          Returns:
            list[tuple]: A list containing tuples with procedure data. Each tuple follows this format: (claim_submission_id, list of cpt_codes followed by their modifiers (e.g. 99214-{{95}}), place_of_service_code, revenue_code, insurance_payout, smartpay_pr)
         """,
    ),
    Tool(
        name="Lookup Approved Claims where Procedure with Modifier was approved",
        func=lookup_approved_claims_with_modifier,
        description="""
          Useful to find claims where a certain modifier was approved. 
          You can use the claim_submission_id to determine which procedures that are in the same claim.
          
          Args: 
            modifier (str): the modifier you want to search for (e.g. 90)
          Returns:
            list[tuple]: A list containing tuples with procedure data. Each tuple follows this format: (claim_submission_id, list of cpt_codes followed by their modifiers (e.g. 99214-{{95}}), place_of_service_code, revenue_code, insurance_payout, smartpay_pr)
         """,
    ),
]

In [206]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from typing import Optional, Type, Any

class ResubmissionsByRarcCarcDf:
  def __init__(self, verbose=True):
    # TODO: need to refresh this data every 24 hours or so
    self.resubmissions_by_rarc_carc_df = pd.read_csv("./carc_rarc_resubmissions.csv")

    self.agent = create_pandas_dataframe_agent(
        ChatOpenAI(model_name='gpt-4-32k', temperature=0),
        self.resubmissions_by_rarc_carc_df,
        max_iterations=5,
        verbose=verbose,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
  
  def run(self, query):
    return self.agent.run(query)


class ResubmissionsByRarcCarcDfTool(BaseTool):
  name = "Ask Questions about Resubmissions for Similar RARCs & CARCs"
  description = """"
    Ask questions about a dataset with resubmissions for similar RARCs & CARCs.

    The dataset has the following columns:
    - reconciliation_owner_status: can be ATHELAS_RESPONSIBILITY (this is the default status for all claims, indicates that it is our responsibility to investigate and/or fix), NON_WORKABLE (this indicates a human has determined we cannot fix this denial), or SITE_RESPONSIBILITY (this indicates a human has determined we need more information from the provider to fix this denial)
    - remittance_advice_remark_code
    - remittance_advice_remark_code_description
    - claim_adjustment_reason_code
    - claim_adjustment_reason_code_description
    - no_approved_resubmissions: number of approved resubmissions
    - list_approved_resubmissions: list of approved resubmissions
    - no_rejected_resubmissions: number of rejected resubmissions
    - list_rejected_resubmissions: list of rejected resubmissions
    - no_denied_resubmissions: number of denied resubmissions
    - list_denied_resubmissions: list of denied resubmissions
    - no_partially_denied_resubmissions: number of partially denied resubmissions
    - list_partially_denied_resubmissions: list of partially denied resubmissions
    - unworked_claims_count: number of denied claims that have not been resubmitted
    - unworked_claims_list: list of denied claims that have not been resubmitted
  """

  def _run(self, query: str):
    safety = "Ensure that valid json is used with internal tools and functions. All columns in the dataframe are strings. Also, outputs should be in well formatted JSON. Also please check to ensure that we stay within the model's context limit."
    agent = ResubmissionsByRarcCarcDf()
    return agent.run("Please use this natural language prompt to select up to 20 results:" + query + safety)

async def _arun(
      self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
  ) -> str:
      """Use the tool asynchronously."""
      raise NotImplementedError("ResubmissionsByRarcCarcDfTool tool does not support async")

In [207]:

from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from typing import Optional, Type, Any


class RarcCarcResubmissionsQA:
  def __init__(self, verbose=True):
    self.tools = [ 
      *shared_agent_tools,
      ResubmissionsByRarcCarcDfTool()
    ]
    
    self.llm = OpenAI(temperature=0, model_name="gpt-4-32k")

    PREFIX = """
      You are an AI assistant that is helping a medical biller understand how certain denial groups were handled before. \
      Denial groups are defined by the RARC and CARC codes, which are returned by the insurance company when a claim is denied. \
      It's usually helpful to first identify claims that were affected by similar denial groups and understand the changes that were made to those claims. \
    """
    self.agent = initialize_agent(
      self.tools,
      self.llm,
      agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
      agent_kwargs={ 'prefix': PREFIX },
      verbose=True
    )
  
  def run(self, query):
    return self.agent.run(query)


class RarcCarcResubmissionsQATool(BaseTool):
  name = "Ask Questions about Resubmissions for RARCs & CARCs"
  description = """"
    Input should be a fully-formed questions about historical resubmissions for certain RARCs and/or CARCs
    Will return a summarized answer with specific claim ids when applicable.

    Here are some examples of what you can ask:
    - What changes where usually made for claims that were denied with RARC 96 and CARC 24, and then resubmitted and approved?
    - Did updating the provider's taxonomy code usually fix denials with RARC 96 and CARC 24?
    - What fixes did not work for claims that were denied with RARC 96 and CARC 24?
    - Do we usually mark claims with RARC 96 and CARC 24 as NON WORKABLE OR SITE RESPONSIBILITY?
  """

  def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Run the agent."""
        agent = RarcCarcResubmissionsQA()
        return agent.run(query)

  async def _arun(
      self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
  ) -> str:
      """Use the tool asynchronously."""
      raise NotImplementedError("RarcCarcResubmissionsTool tool does not support async")

In [208]:
# hr = RarcCarcResubmissionsQATool()
# hr.run("How are denials due to RARC N286 usually handled?")

In [209]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from typing import Optional, Type, Any

class PatientInsuranceInfo:
  def __init__(self):
    self.tools = [
      *shared_agent_tools, 
      Tool(
          name="Lookup Summary of Patient Insurance History for Claim",
          func=summarize_patient_insurance_history_for_claim,
          description="""
              Useful for when you need to understand the history of a patient's policy/coverage.

              Args:
                claim_id (int): the id of the claim you want to lookup
              Returns:
                str: summary of the patient's insurance history
          """,
        ), 
    ]
    
    self.llm = OpenAI(temperature=0, model_name="gpt-4-32k")
    self.agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


  def run(self, query: str):
    safety = "\n Please check to ensure that we stay within the model's context limit."
    return self.agent.run(query + safety)


class PatientInsuranceInfoTool(BaseTool):
  name = "Ask Questions about Patient Insurance Info"
  description = """"
    Useful for when you need to ask questions about a patient's insurance info.
  """

  def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        
        pii = PatientInsuranceInfo()
        tool_output = pii.run(query)
        return tool_output

  async def _arun(
      self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
  ) -> str:
      """Use the tool asynchronously."""
      raise NotImplementedError("PatientInsuranceInfo tool does not support async")

In [210]:
# pii_tool = PatientInsuranceInfoTool()
# pii_tool.run("Was the patient's insurance active on 07/12/2023 for claim 480701?")

In [211]:
# import pandas as pd

# def get_definition_for_modifier(modifier: str):
#   if not isinstance(modifier, str):
#     modifier = str(modifier)
#   modifier = modifier.upper()

#   df = pd.read_csv("./all_modifiers.csv")
#   df = df[df["Modifier"] == modifier]
#   return json.dumps(json.loads(df.to_json(orient="records")), indent=4)

In [212]:
# get_definition_for_modifier(modifier=90)

In [213]:
# print(summarize_patient_insurance_history_for_claim(claim_id=617417))

In [219]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    *shared_agent_tools,
    RarcCarcResubmissionsQATool()
    # PatientInsuranceInfoTool()
]

PREFIX = """ 
  You are an AI assistant that is trying to solve medical billing claims that were denied by insurers. 
  You will be given a claim submission that was denied. 
  First, use the provided tools to understand why the claim was denied. There may be multiple procedures in a claim, so analyze the procedures with 0 insurance_payout and understand their denial reasons.
  Use all the tools to provide a very specific solution to the provided claim.
  Usually, you can get hints to solve a claim by looking at the RARC/CARC description and claims denied with similar RARC and CARC codes. You can use this information to decide what information you need to gather and/or update for the denied claim.
  Claims have multiple procedure. Each procedure can have separate and multiple denial reasons. You should try to investigate one denial reason at a time.
"""
llm = OpenAI(temperature=0, model_name="gpt-4-32k")
denials_copilot = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, agent_kwargs={ 'prefix': PREFIX }, verbose=True)

In [ ]:
# TODO: each tool can have its own agent? - https://github.com/jbpayton/langchain-stock-screener/blob/210f13daa70925504a782570afb4c9e0a54786da/alpaca_stock_screener/StockScreener.py -> use pandas dataframe agent
# --- pandas dataframe agent -> https://github.com/langchain-ai/langchain/blob/65e1606daa696e2190fcb410f190c6811f9f8dc3/libs/langchain/langchain/agents/agent_toolkits/pandas/base.py#L27
# TODO: use pandas dataframe agent to group by claim submissions - https://python.langchain.com/docs/integrations/toolkits/pandas
# TODO: using memory? 
# TODO: use google to searh for modifiers description
# TODO: add more context to each claim submission: provider details, etc.
# TODO: add more context on what rules are being applied
# TODO: show diffs between resubmissions that were successful and the one that failed
# TODO: track token usage throughout the conversation
# TODO: offline job that gets latest rarc_carc combos and unworked and approved examples
# TODO: show denied/rejected resubmissions for each rarc/carc combo so it can understand what did not work
# TODO: start by indentifying what context about the claim to pull in based on other rarc/carc solutions
# TODO: claims in rarc/carc combos that were resubmitted and approved -> may have other changes made due to a different rarc/carc combo on the same claim -> how to get the model to ignore these changes?
# TODO: need to feed provider taxonomy code (GP modifiers, etc.)
# TODO: add more context to explain reconconciliation owner status -> mark things as NW or SR to collect more data
# TODO: add tool 
# TODO: add more context to explain payer mapping + rules
# TODO: see if we can start suggesting payer mapping + rule changes
# TODO: add tool to query npi API
# TODO: add tool to gather more info about the site (site info stuff like addresses, facilities, providers, etc.) -> tool w/ agent
# TODO: how we can verify what a patient's insurance plan covers? specific procedures, etc.
# TODO: add pg-vector support to our DB

# Testing
# TODO: set up langsmith dataset + benchmarking - https://python.langchain.com/docs/guides/langsmith/walkthrough

# Deployment
# TODO: need to create data modal for how copilot results are stored in the DB (store timestamp, entire response, claim_id, etc.)
# TODO: need to figure out how to use errands



# High-level
# TODO: need to update architecture to use routers and process each rarc/carc combo independently????
# TODO: update tools to take more general inquiries and use their own agents to answer questions
# TODO: focus on how to incorporate rules from resubmissions: look at trigger conditions + actions
# TODO: focus on what context seems important to bring into to the claim: cpt codes, modifiers, provider info, taxonomy, payer mapping, etc. 
# TODO: see if we can embed the denials training in a vector store and then use that to find helpful steps/info that we need to debug the denials - https://python.langchain.com/docs/modules/data_connection/vectorstores/
# ----> notion example: https://github.com/hwchase17/notion-qa/blob/master/qa.py
# TODO: see if we can run use a similar process as map re-rank: run the LLM Multiple times on the same denial and have it assign a confidence score -> choose the one with the higest confidence score -> https://python.langchain.com/docs/modules/chains/document/map_rerank

In [220]:
# Misrouted claim
# Missing CLIA
# Missing EOB
# Non-covered charges
denials_copilot.run("How can I fix claim 480506?")



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will use the Lookup Claim Procedure Denial Info tool to get the denial reasons for each procedure in the claim.
Action: Lookup Claim Procedure Denial Info
Action Input: 480506
Observation: [
    {
        "claim_submission_id": 480506,
        "cpt_code": "G0439",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "96 - Non-covered charge(s). At least one Remark Code must be provided (may be comprised of either the NCPDP Reject Reason Code, or Remittance Advice Remark Code that is not an ALERT.) Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            "N431 - Not covered with this procedure."
        ]
  

'Add the CLIA certification number to the claim and resubmit it. If the claim is still denied, contact the payer to understand why the procedures with cpt_codes "G0439" and "36415" were denied and what information or conditions are necessary for these procedures to be approved.'

In [123]:
denials_copilot.run("How can I fix claim 617417?")



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will do this by looking up the claim procedure denial info.
Action: Lookup Claim Procedure Denial Info
Action Input: 617417
Observation: [
    {
        "claim_submission_id": 617417,
        "cpt_code": "99214",
        "modifiers": [
            "95",
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 130.0,
        "claim_adjustment_reason_codes": [
            "27 - Expenses incurred after coverage terminated."
        ],
        "remittance_advice_remark_codes": [
            "N650 - This policy was not in effect for this date of loss. No coverage is available."
        ]
    },
    {
        "claim_submission_id": 617417,
        "cpt_code": "90836",
        "modifiers": [
            "95"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insuranc

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Services not provided by network/primary care providers.
denials_copilot.run("How can I fix claim 191501?")



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 191501.
Action: Lookup Claim Procedure Denial Info
Action Input: 191501
Observation: [
    {
        "claim_submission_id": 191501,
        "cpt_code": "99396",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 239.12,
        "claim_adjustment_reason_codes": [
            "242 - Services not provided by network/primary care providers."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    }
]
Thought:The claim was denied due to reason code 242 - Services not provided by network/primary care providers. I need to understand what this CPT code 99396 is for.
Action: Get CPT Code Description
Action Input: 99396
Observation: [
    {
        "cpt_code": "99396",
        "description": "Prev visit, est, age 40-64"

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


To answer this question, I need to first identify the claims that were denied with CARC 242 and then resubmitted and approved. Then, I can look up the changes made to these resubmitted claims.
Action: Ask Questions about Resubmissions for Similar RARCs & CARCs
Action Input: {"claim_adjustment_reason_code": "242"}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['claim_adjustment_reason_code'] == '242'].head(20).to_json(orient='records')"}`


[{"reconciliation_owner_status":"ATHELAS_RESPONSIBILITY","remittance_advice_remark_code":"M115","remittance_advice_remark_code_description":"This item is denied when provided to this patient by a non-contract or non-demonstration supplier.","claim_adjustment_reason_code":"242","claim_adjustment_reason_code_description":"Services not provided by network\/primary care providers.","no_approved_resubmissions":0,"list_approved_resubmissions":"{null}","no_rejected_resubmissions":0,"list_rejected_resubmissions":"{

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(



Observation: For the resubmitted claim ID 407678, the member ID was changed from '10792097600' to '107920976'. In the resubmitted claim ID 407724, the member ID was updated from '10792097601' to '107920976'. For the resubmitted claim ID 441268, the organization name was corrected by removing an extra space at the end, and the line item charge amount was increased from '130' to '376.05'.
Thought:I now know the changes that were made to the resubmitted claims.
Final Answer: The changes made to the resubmitted claims that were initially denied with CARC 242 and then approved are as follows: For claim ID 407678, the member ID was changed from '10792097600' to '107920976'. In claim ID 407724, the member ID was updated from '10792097601' to '107920976'. For claim ID 441268, the organization name was corrected by removing an extra space at the end, and the line item charge amount was increased from '130' to '376.05'.

> Finished chain.

Observation: The changes made to the resubmitted claims

'Append the modifier 25 to the CPT code 99396 and resubmit the claim.'

In [102]:
# Frequency exceeded benefits max
question = "How should I fix claim 452559?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 452559.
Action: Lookup Claim Procedure Denial Info
Action Input: 452559
Observation: [
    {
        "claim_submission_id": 452559,
        "cpt_code": "99214",
        "modifiers": [
            "95",
            "kx"
        ],
        "place_of_service_code": "02",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "119 - Benefit maximum for this time period or occurrence has been reached."
        ],
        "remittance_advice_remark_codes": [
            "N640 - Exceeds number/frequency approved/allowed within time period."
        ]
    }
]
Thought:The claim was denied because the benefit maximum for this time period or occurrence has been reached (CARC 119) and it exceeds the number/frequency approved/allowed within time period (RARC N64

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


To answer this question, I need to first find the claims that were denied with RARC N640 and CARC 119, and then resubmitted and approved. I can do this by using the "Ask Questions about Resubmissions for Similar RARCs & CARCs" function.
Action: Ask Questions about Resubmissions for Similar RARCs & CARCs
Action Input: {"remittance_advice_remark_code": "N640", "claim_adjustment_reason_code": "119"}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['remittance_advice_remark_code'] == 'N640') & (df['claim_adjustment_reason_code'] == '119')].head(20).to_json(orient='records')"}`


[{"reconciliation_owner_status":"ATHELAS_RESPONSIBILITY","remittance_advice_remark_code":"N640","remittance_advice_remark_code_description":"Exceeds number\/frequency approved\/allowed within time period.","claim_adjustment_reason_code":"119","claim_adjustment_reason_code_description":"Benefit maximum for this time period or occurrence has been reached.","no_approved_resub

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(



Observation: 1) For resubmitted claim ID 170849, the receiver's organization name was changed from 'NEVADA BLUE SHIELD' to 'ANTHEM KY MEDICAID' and the trading partner name and service ID were also updated accordingly. The service lines remained the same but the patient control number was updated.

2) In the resubmitted claim ID 428358, the billing organization name was slightly modified and the subscriber's address and member ID were updated. The service lines were adjusted with a change in procedure modifiers and a reduction in the line item charge amount. The place of service code was also changed from '02' to '10' and the service facility location was updated.
Thought:I now know the changes that were made to the claims that were denied with RARC N640 and CARC 119, and then resubmitted and approved.
Final Answer: For the resubmitted claim ID 170849, the receiver's organization name was changed from 'NEVADA BLUE SHIELD' to 'ANTHEM KY MEDICAID' and the trading partner name and servic

"To fix claim 452559, you should review and potentially update the receiver's organization name, trading partner name, service ID, patient control number, billing organization name, subscriber's address, member ID, procedure modifiers, line item charge amount, place of service code, and service facility location. Also, review the frequency of the procedure 99214 for this patient to ensure it does not exceed the allowed frequency within the time period."

In [105]:
# time filing exceeded - works here
question = "How should I fix claim 14857?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 14857.
Action: Lookup Claim Procedure Denial Info
Action Input: 14857
Observation: [
    {
        "claim_submission_id": 14857,
        "cpt_code": "U0003",
        "modifiers": [],
        "place_of_service_code": "81",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "29 - The time limit for filing has expired."
        ],
        "remittance_advice_remark_codes": [
            "N211 - Alert: You may not appeal this decision."
        ]
    },
    {
        "claim_submission_id": 14857,
        "cpt_code": "U0005",
        "modifiers": [],
        "place_of_service_code": "81",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "29 - The time lim

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


To answer this question, I need to first find the claims that were denied with RARC N211 and CARC 29, and then resubmitted and approved. Then, I can look up the changes made to these resubmitted claims.
Action: Ask Questions about Resubmissions for Similar RARCs & CARCs
Action Input: {"remittance_advice_remark_code": "N211", "claim_adjustment_reason_code": "29"}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['remittance_advice_remark_code'] == 'N211') & (df['claim_adjustment_reason_code'] == '29')].head(20).to_json(orient='records')"}`


[{"reconciliation_owner_status":"ATHELAS_RESPONSIBILITY","remittance_advice_remark_code":"N211","remittance_advice_remark_code_description":"Alert: You may not appeal this decision.","claim_adjustment_reason_code":"29","claim_adjustment_reason_code_description":"The time limit for filing has expired.","no_approved_resubmissions":0,"list_approved_resubmissions":null,"no_rejected_resubmissions":0,"list_rejecte

'The claim 14857 was denied due to being submitted after the deadline set by the insurance company. The remittance advice remark code "N211 - Alert: You may not appeal this decision." suggests that this decision cannot be appealed. There are no cases where a claim denied for the same reason was resubmitted and approved. Therefore, it seems that the claim cannot be fixed and resubmitted.'

In [106]:
# do not need 25 modifier on 96372 (no other E/M service on claim) -> works
question = "How should I fix claim 226518?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 226518.
Action: Lookup Claim Procedure Denial Info
Action Input: 226518
Observation: [
    {
        "claim_submission_id": 226518,
        "cpt_code": "96372",
        "modifiers": [
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    },
    {
        "claim_submission_id": 226518,
        "cpt_code": "J3301",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
       

'To fix the claim, remove the modifier 25 from the procedure 96372 since it was not performed in conjunction with another procedure.'

In [ ]:
#  missing patient medical records
question = "How should I fix claim 43434?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the denial information for the procedures in the claim.
Action: Lookup Claim Procedure Denial Info
Action Input: 43434
Observation: [
    {
        "claim_submission_id": 43434,
        "cpt_code": "97140",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "252 - An attachment/other documentation is required to adjudicate this claim/service. At least one Remark Code must be provided (may be comprised of either the NCPDP Reject Reason Code, or Remittance Advice Remark Code that is not an ALERT).",
            "252 - An attachment/other documentation is required to adjudicate this claim/service. At least one Remark Code must be provided (may be comprised of either the NCPDP Reject Reason Code, or Remittance Advice Remark Co

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(



Observation: Previous Claim: ID 12345, Procedure Code: 99213, Diagnosis Code: M54.5, Provider: Dr. Smith, Amount: $150
Resubmitted Claim: ID 67890, Procedure Code: 99214, Diagnosis Code: M54.6, Provider: Dr. Smith, Amount: $200

The resubmitted claim ID 67890 has a different procedure code (99214 instead of 99213) and diagnosis code (M54.6 instead of M54.5) compared to the previous claim. Additionally, the amount charged has increased from $150 to $200.
Thought:The changes made to the claim payload when it was resubmitted and approved include a change in the procedure code, diagnosis code, and the amount charged. This suggests that the claim might have been denied initially due to incorrect procedure or diagnosis codes, or the amount charged. Therefore, the solution to fix the claim would be to provide the missing documentation and patient medical records, resubmit the claim with the correct procedure and diagnosis codes, and adjust the amount charged if necessary.
Final Answer: To fi

'To fix claim 43434, you should provide the missing documentation and patient medical records, resubmit the claim with the correct procedure and diagnosis codes (99214 and M54.6 respectively), and adjust the amount charged if necessary.'

In [108]:
# incorrectly added 95 modifier on two injection codes (96372 and J1071) -> works
question = "How should I fix claim 276585?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 276585.
Action: Lookup Claim Procedure Denial Info
Action Input: 276585
Observation: [
    {
        "claim_submission_id": 276585,
        "cpt_code": "96372",
        "modifiers": [
            "95"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            "N519 - Invalid combination of HCPCS modifiers."
        ]
    },
    {
        "claim_submission_id": 276585,
        "cpt_code": "99213",
        "modifiers": [
            "25",
            "95"


'Resubmit the claim with the procedure codes 96372 and J1071 without the modifier 95 or with the modifier 25.'

In [109]:
# the claim should add 25 modifier to 90833, and should remove 95 modifier from 99214 (look at POS)
question = "How should I fix claim 518796?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 518796.
Action: Lookup Claim Procedure Denial Info
Action Input: 518796
Observation: [
    {
        "claim_submission_id": 518796,
        "cpt_code": "90833",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    },
    {
        "claim_submission_id": 518796,
        "cpt_code": "99214",
        "modifiers": [
            "25",
            "95"
        ],
        "place_of_service_code": "2",
        "revenue_cod

'The claim should be resubmitted with the modifier 95 for the procedure with the CPT code 90833.'

In [111]:
# COMPLETELY WRONG
question = "How should I fix claim 544924?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 544924.
Action: Lookup Claim Procedure Denial Info
Action Input: 544924
Observation: [
    {
        "claim_submission_id": 544924,
        "cpt_code": "90868",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    },
    {
        "claim_submission_id": 544924,
        "cpt_code": "99214",
        "modifiers": [
            "95",
            "25"
        ],
        "place_of_service_code": "2",
        "revenue_cod

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


To answer this question, I need to first identify the claims that were denied with CARC 4, then find out what changes were made when these claims were resubmitted and approved.
Action: Ask Questions about Resubmissions for Similar RARCs & CARCs
Action Input: {"claim_adjustment_reason_code": "4"}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['claim_adjustment_reason_code'] == '4'].head(20).to_json(orient='records')"}`


[{"reconciliation_owner_status":"SITE_RESPONSIBILITY","remittance_advice_remark_code":"M20","remittance_advice_remark_code_description":"Missing\/incomplete\/invalid HCPCS.","claim_adjustment_reason_code":"4","claim_adjustment_reason_code_description":"The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.","no_approved_resubmissions":0,"list_approved_resubmissions":"{null}","no_rejected_resubmissions":

KeyboardInterrupt: 

In [112]:
# Was correct to say we need to remove the 90 and 25 modifiers from the claim
# But it was not sure what to do about the non-covered charges???
question = "How should I fix claim 194208?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 194208.
Action: Lookup Claim Procedure Denial Info
Action Input: 194208
Observation: [
    {
        "claim_submission_id": 194208,
        "cpt_code": "99214",
        "modifiers": [
            "25",
            "90"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            "N644 - Reimbursement has been made according to the bilateral procedure rule."
        ]
    },
    {
        "claim_submission_id": 194208,
        "cpt_code": "36415",
        "

"For the procedure with the CPT code 99214, remove the modifiers 25 and 90. For the procedure with the CPT code 36415, check with the patient's insurance plan to see if the procedure is covered. If it is covered, then it might be necessary to provide additional documentation to justify the necessity of the procedure."

In [113]:
# Should remove 59 modifier from 99214
# started looking at rarc-carc resubmissions -> need to add router here
question = "How should I fix claim 219277?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial information.
Action: Lookup Claim Procedure Denial Info
Action Input: 219277
Observation: [
    {
        "claim_submission_id": 219277,
        "cpt_code": "99396",
        "modifiers": [
            "59",
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            "N644 - Reimbursement has been made according to the bilateral procedure rule."
        ]
    },
    {
        "claim_submission_id": 219277,
        "cpt_code": "96127",
        "modifiers"

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


To answer this question, I need to first find the claims that were denied with RARC N644 and CARC 4, and then resubmitted and approved. I can do this by using the "Ask Questions about Resubmissions for Similar RARCs & CARCs" function. Once I have the list of these claims, I can then use the "Lookup Changes Made to Resubmitted Claim Payload" function to find out what changes were made to these claims.
Action: Ask Questions about Resubmissions for Similar RARCs & CARCs
Action Input: {"remittance_advice_remark_code": "N644", "claim_adjustment_reason_code": "4"}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['remittance_advice_remark_code'] == 'N644') & (df['claim_adjustment_reason_code'] == '4')].head(20).to_json(orient='records')"}`


[{"reconciliation_owner_status":"ATHELAS_RESPONSIBILITY","remittance_advice_remark_code":"N644","remittance_advice_remark_code_description":"Reimbursement has been made according to the bilateral procedure rule."

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(



Observation: The resubmitted claim with ID 587235 shows a change in the procedure modifier from '93' to '95' for the professional service with procedure code '99213'. Additionally, the place of service code has been updated from '11' to '02'.
Thought:I now know the changes that were made to the claim that was initially denied with RARC N644 and CARC 4, and then resubmitted and approved.
Final Answer: The claim that was initially denied with RARC N644 and CARC 4, and then resubmitted and approved, had a change in the procedure modifier from '93' to '95' for the professional service with procedure code '99213'. Additionally, the place of service code was updated from '11' to '02'.

> Finished chain.

Observation: The claim that was initially denied with RARC N644 and CARC 4, and then resubmitted and approved, had a change in the procedure modifier from '93' to '95' for the professional service with procedure code '99213'. Additionally, the place of service code was updated from '11' to 

'To fix the claim, remove the modifier 59 from the procedure with the CPT code 99396. No changes are needed for the procedures with the CPT codes 96127 and 17110.'

In [114]:
# incorrectly added rule to submit claims with GP modifier -> some payers seem to accidentally approve this, so the model is picking up that GP is valid for 97803
# the correct solution is to remove the GP modifier
# Need to show the model that a bunch of claims w/ this issue were fixed by removing the GP modifier
question = "How should I fix claim 583308?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will lookup the claim procedure denial info for claim 583308.
Action: Lookup Claim Procedure Denial Info
Action Input: 583308
Observation: [
    {
        "claim_submission_id": 583308,
        "cpt_code": "97803",
        "modifiers": [
            "GP",
            "95"
        ],
        "place_of_service_code": "10",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.",
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_cod

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


To answer this question, I need to first identify the claims that were denied with CARC 4, then find out what changes were made when these claims were resubmitted and approved.
Action: Ask Questions about Resubmissions for Similar RARCs & CARCs
Action Input: {"claim_adjustment_reason_code": "4"}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['claim_adjustment_reason_code'] == '4'].head(20).to_json(orient='records')"}`


[{"reconciliation_owner_status":"SITE_RESPONSIBILITY","remittance_advice_remark_code":"M20","remittance_advice_remark_code_description":"Missing\/incomplete\/invalid HCPCS.","claim_adjustment_reason_code":"4","claim_adjustment_reason_code_description":"The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.","no_approved_resubmissions":0,"list_approved_resubmissions":"{null}","no_rejected_resubmissions":

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(



Observation: 1. For resubmitted claim ID 205967, the taxonomy code for billing and rendering was changed from '363LP2300X' to '363LF0000X'. The receiver organization was changed from 'CONNECTICUT MEDICAID' to 'UNITED HEALTHCARE' and the service facility location NPI was updated from '1962400150' to '1306488762'.

2. In claim ID 205964, the taxonomy code for billing and rendering was updated from '163WC1500X' to '363LF0000X'. The receiver organization was changed from 'CONNECTICUT MEDICAID' to 'UNITED HEALTHCARE' and the service facility location NPI was updated from '1598065245' to '1306488762'.

3. For claim ID 205970, the taxonomy code for billing and rendering was updated from '163WC1500X' to '363LF0000X'. The receiver organization was changed from 'CONNECTICUT MEDICAID' to 'UNITED HEALTHCARE' and the service facility location NPI was updated from '1962400150' to '1306488762'.

4. In claim ID 223189, the place of service code was changed from '02' to '11'. 

5. For claim ID 205973,

'The solution to this claim could be to remove the modifier GP from the procedure code 97803 and resubmit the claim.'

In [ ]:
# Example 8
question = "How should I fix claim 58653?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will look up the claim procedure denial info for claim 58653.
Action: Lookup Claim Procedure Denial Info
Action Input: 58653
Observation: [
    {
        "claim_submission_id": 58653,
        "cpt_code": "82962",
        "modifiers": [
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "16 - Claim/service lacks information or has submission/billing error(s). Usage: Do not use this code for claims attachment(s)/other documentation. At least one Remark Code must be provided (may be comprised of either the NCPDP Reject Reason Code, or Remittance Advice Remark Code that is not an ALERT.) Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advi

'To fix claim 58653, consider including additional procedures such as "99214", "81025", "81002", "93000", and "36415" with the cpt_code "82962". Also consider adding the modifier "QW" with the cpt_code "82962". For the procedure with cpt_code "99214", consider adding the modifier "95". Ensure that all necessary information is provided in a timely and complete manner.'

In [ ]:
# Example 8
question = "How should I fix claim 92887?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
First, I need to understand why the claim was denied. I will do this by looking up the claim procedure denial info for claim 92887.
Action: Lookup Claim Procedure Denial Info
Action Input: 92887
Observation: [
    {
        "claim_submission_id": 92887,
        "cpt_code": "87637",
        "modifiers": [],
        "place_of_service_code": "81",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "16 - Claim/service lacks information or has submission/billing error(s). Usage: Do not use this code for claims attachment(s)/other documentation. At least one Remark Code must be provided (may be comprised of either the NCPDP Reject Reason Code, or Remittance Advice Remark Code that is not an ALERT.) Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.",
            "16 - Claim/service lacks informat

'The claim should be resubmitted with the correct information and possibly include additional procedures that are often associated with the cpt_codes in the claim.'